## Differential cross section for $e^+ e^- \rightarrow \mu^+ \mu^-$ 

### Introduction

The goal of this tutorial is to compute the differential cross section for the process $e^+ e^- \rightarrow \mu^+ \mu^-$ using __helicity amplitudes__. 

In the
__Dirac representation__ of the gamma matrices, the free particle solutions of the Dirac equation (1928)

\begin{align}
    (i \gamma^\mu \partial_\mu - m) \psi = 0,\quad
    \gamma^\mu \gamma^\nu + \gamma^\nu \gamma^\mu = 2 g^{\mu\nu},
\end{align}
are
\begin{align}
    u_r(p) & = \sqrt{E + m} \begin{pmatrix} \phi_r \\ \frac{\mathbf{\sigma} \cdot \mathbf{p}}{E + m} \phi_r \end{pmatrix},\\
  v_r(p) & = \sqrt{E + m} \begin{pmatrix} \frac{\mathbf{\sigma} \cdot \mathbf{p}}{E + m} \chi_r \\ \chi_r \end{pmatrix},
\end{align}
for particles and antiparticles, respectively;  $\sigma$ are the $2\times 2$ Pauli matrices.
The 2-component spinors are chosen to be
\begin{align}
    \phi_1 & = \chi_2 = \begin{pmatrix} 1 \\ 0 \end{pmatrix},\\
    \phi_2 & = \chi_1 = \begin{pmatrix} 0 \\ 1 \end{pmatrix},
\end{align}


Tip: To disable a cell use __esc r__; use __esc y__ to reactivate it and __esc m__ to go to markdown mode.

In [1]:
# load symbolic algebra module
import sympy as sm
import sympy.functions.special.tensor_functions as t
from sympy.matrices import Matrix

# gamma matrices and sigma (Pauli) matrices
from sympy.physics.matrices import mgamma, msigma

# complex number 0 + I and simplifiers
from sympy import I, trigsimp, radsimp

# symbolic solver
from sympy.solvers import solve

# enable pretty printing of symbolic equations
from IPython.display import display
sm.init_printing(use_latex='mathjax')

levi = t.LeviCivita
kron = t.KroneckerDelta

### Standard Matrices

$\gamma^0, \gamma^1, \gamma^2, \gamma^3$ and Pauli matrices $\sigma_i$.

$\gamma^5 = i \gamma^0 \gamma^1 \gamma^2 \gamma^3$

$P_L = (1 - \gamma^5)/2$

$P_R = (1 + \gamma^5)/2$

Note: $\gamma^5 \gamma^5 = 1$.

In [2]:
G0  = mgamma(0)
G1  = mgamma(1)
G2  = mgamma(2)
G3  = mgamma(3)
G5  = mgamma(5)

ONE = G5*G5

PL  = (ONE - G5)/2
PR  = (ONE + G5)/2

sigma1 = msigma(1)
sigma2 = msigma(2)
sigma3 = msigma(3)

G0,G1,G2,G3

⎛⎡1  0  0   0 ⎤  ⎡0   0   0  1⎤  ⎡0   0  0  -ⅈ⎤  ⎡0   0  1  0 ⎤⎞
⎜⎢            ⎥  ⎢            ⎥  ⎢            ⎥  ⎢            ⎥⎟
⎜⎢0  1  0   0 ⎥  ⎢0   0   1  0⎥  ⎢0   0  ⅈ  0 ⎥  ⎢0   0  0  -1⎥⎟
⎜⎢            ⎥, ⎢            ⎥, ⎢            ⎥, ⎢            ⎥⎟
⎜⎢0  0  -1  0 ⎥  ⎢0   -1  0  0⎥  ⎢0   ⅈ  0  0 ⎥  ⎢-1  0  0  0 ⎥⎟
⎜⎢            ⎥  ⎢            ⎥  ⎢            ⎥  ⎢            ⎥⎟
⎝⎣0  0  0   -1⎦  ⎣-1  0   0  0⎦  ⎣-ⅈ  0  0  0 ⎦  ⎣0   1  0  0 ⎦⎠

In [3]:
G5, PL, PR

⎛⎡0  0  1  0⎤  ⎡1/2    0    -1/2   0  ⎤  ⎡1/2   0   1/2   0 ⎤⎞
⎜⎢          ⎥  ⎢                      ⎥  ⎢                  ⎥⎟
⎜⎢0  0  0  1⎥  ⎢ 0    1/2    0    -1/2⎥  ⎢ 0   1/2   0   1/2⎥⎟
⎜⎢          ⎥, ⎢                      ⎥, ⎢                  ⎥⎟
⎜⎢1  0  0  0⎥  ⎢-1/2   0    1/2    0  ⎥  ⎢1/2   0   1/2   0 ⎥⎟
⎜⎢          ⎥  ⎢                      ⎥  ⎢                  ⎥⎟
⎝⎣0  1  0  0⎦  ⎣ 0    -1/2   0    1/2 ⎦  ⎣ 0   1/2   0   1/2⎦⎠

In [4]:
sigma1, sigma2, sigma3

⎛⎡0  1⎤  ⎡0  -ⅈ⎤  ⎡1  0 ⎤⎞
⎜⎢    ⎥, ⎢     ⎥, ⎢     ⎥⎟
⎝⎣1  0⎦  ⎣ⅈ  0 ⎦  ⎣0  -1⎦⎠

### Define symbols

Some symbols are defined to be non-commutative simply to preserve their order in expressions! We define symbolic versions of the above matrices or $2\times 2$ versions of them where it is understood that terms such as $1$ or $0$ are really $2\times 2$ matrices.

In [5]:
p    = sm.Symbol('p',   commutative=True)
px   = sm.Symbol('p_x', commutative=True)
py   = sm.Symbol('p_y', commutative=True)
pz   = sm.Symbol('p_z', commutative=True)
m    = sm.Symbol('m', commutative=True)
E    = sm.Symbol('E', commutative=True)

A    = sm.Symbol('A', commutative=False)
B    = sm.Symbol('B', commutative=False)

sp   = sm.Symbol('\sigma\cdot p', commutative=False)
sigma= sm.Symbol('\sigma',   commutative=False)
sigm1= sm.Symbol('\sigma_1', commutative=False)
sigm2= sm.Symbol('\sigma_2', commutative=False)
sigm3= sm.Symbol('\sigma_3', commutative=False)

phi  = sm.Symbol('\phi', commutative=False)
phi_dagger= sm.Symbol('\phi^\dagger', commutative=False)

chi  = sm.Symbol('\chi', commutative=False)
chi_dagger= sm.Symbol('\chi^\dagger', commutative=False)

phi_1       = sm.Symbol('\phi_1', commutative=False)
phi_1_dagger= sm.Symbol('\phi_1^\dagger', commutative=False)
phi_2       = sm.Symbol('\phi_2', commutative=False)
phi_2_dagger= sm.Symbol('\phi_2^\dagger', commutative=False)

chi_1       = sm.Symbol('\chi_1', commutative=False)
chi_1_dagger= sm.Symbol('\chi_1^\dagger', commutative=False)
chi_2       = sm.Symbol('\chi_2', commutative=False)
chi_2_dagger= sm.Symbol('\chi_2^\dagger', commutative=False)

g0 = Matrix([[1, 0], [0,-1]])
g1 = Matrix([[0, sigm1],[-sigm1, 0]])
g2 = Matrix([[0, sigm2],[-sigm2, 0]])
g3 = Matrix([[0, sigm3],[-sigm3, 0]])
g5 = Matrix([[0, 1], [1,0]])

U    = sm.sqrt(E + m)*Matrix([phi, sp/(E+m)*phi])
V    = sm.sqrt(E + m)*Matrix([sp/(E+m)*chi, chi])

AB   = Matrix([A, B])

one  = Matrix([[1, 0], [0, 1]])

Phi_1= Matrix([[1], [0]])
Phi_2= Matrix([[0], [1]])

Chi_1= Matrix([[0], [1]])
Chi_2= Matrix([[1], [0]])

pL   = (one - sigma1)/2
pR   = (one + sigma1)/2

g0, g1, g2, g3, g5

⎛⎡1  0 ⎤  ⎡   0      \sigma₁⎤  ⎡   0      \sigma₂⎤  ⎡   0      \sigma₃⎤  ⎡0  1
⎜⎢     ⎥, ⎢                 ⎥, ⎢                 ⎥, ⎢                 ⎥, ⎢    
⎝⎣0  -1⎦  ⎣-\sigma₁     0   ⎦  ⎣-\sigma₂     0   ⎦  ⎣-\sigma₃     0   ⎦  ⎣1  0

⎤⎞
⎥⎟
⎦⎠

In [6]:
AB, U, V, pL, pR

⎛     ⎡    _______       ⎤  ⎡\sigma\cdot p⋅\chi⎤                          ⎞
⎜     ⎢  ╲╱ E + m ⋅\phi  ⎥  ⎢──────────────────⎥                          ⎟
⎜⎡A⎤  ⎢                  ⎥  ⎢      _______     ⎥  ⎡1/2   -1/2⎤  ⎡1/2  1/2⎤⎟
⎜⎢ ⎥, ⎢\sigma\cdot p⋅\phi⎥, ⎢    ╲╱ E + m      ⎥, ⎢          ⎥, ⎢        ⎥⎟
⎜⎣B⎦  ⎢──────────────────⎥  ⎢                  ⎥  ⎣-1/2  1/2 ⎦  ⎣1/2  1/2⎦⎟
⎜     ⎢      _______     ⎥  ⎢    _______       ⎥                          ⎟
⎝     ⎣    ╲╱ E + m      ⎦  ⎣  ╲╱ E + m ⋅\chi  ⎦                          ⎠

### $\sigma\cdot\mathbf{p}$ and $\sigma\cdot\hat{\mathbf{p}}$

In [7]:
sigmap = px*sigma1 + py*sigma2 + pz*sigma3
sigmaphat = sigmap / p
sigmap, sigmaphat

⎛                          ⎡   p_z      pₓ - ⅈ⋅p_y⎤⎞
⎜                          ⎢   ───      ──────────⎥⎟
⎜⎡   p_z      pₓ - ⅈ⋅p_y⎤  ⎢    p           p     ⎥⎟
⎜⎢                      ⎥, ⎢                      ⎥⎟
⎜⎣pₓ + ⅈ⋅p_y     -p_z   ⎦  ⎢pₓ + ⅈ⋅p_y    -p_z    ⎥⎟
⎜                          ⎢──────────    ─────   ⎥⎟
⎝                          ⎣    p           p     ⎦⎠

### Left- and right-handed projection operators and chirality operator 
Compute the effect of the left and right-handed projection operators, $P_L = (1 - \gamma^5)/2$ and $P_R = (1 + \gamma^5)/2$, respectively, on the spinor $u = \begin{pmatrix} A \\ B \end{pmatrix}$. Also verify that they are indeed eigen-spinors of the chirality (that is, handedness) operator $\gamma^5$.

Verify

\begin{align}
    u_L & = P_L u, \quad \gamma^5 u_L = -u_L, \\
    u_R & = P_R u, \quad \gamma^5 u_R = +u_R.
\end{align}

In [8]:
UL = pL*AB
UR = pR*AB
AB, UL, g5*UL, UR, g5*UR

⎛     ⎡ A   B ⎤  ⎡  A   B⎤  ⎡A   B⎤  ⎡A   B⎤⎞
⎜     ⎢ ─ - ─ ⎥  ⎢- ─ + ─⎥  ⎢─ + ─⎥  ⎢─ + ─⎥⎟
⎜⎡A⎤  ⎢ 2   2 ⎥  ⎢  2   2⎥  ⎢2   2⎥  ⎢2   2⎥⎟
⎜⎢ ⎥, ⎢       ⎥, ⎢       ⎥, ⎢     ⎥, ⎢     ⎥⎟
⎜⎣B⎦  ⎢  A   B⎥  ⎢ A   B ⎥  ⎢A   B⎥  ⎢A   B⎥⎟
⎜     ⎢- ─ + ─⎥  ⎢ ─ - ─ ⎥  ⎢─ + ─⎥  ⎢─ + ─⎥⎟
⎝     ⎣  2   2⎦  ⎣ 2   2 ⎦  ⎣2   2⎦  ⎣2   2⎦⎠

In [9]:
print(sm.latex(AB))
print(sm.latex(UL))
print(sm.latex(UR))

\left[\begin{matrix}A\\B\end{matrix}\right]
\left[\begin{matrix}\frac{A}{2} - \frac{B}{2}\\- \frac{A}{2} + \frac{B}{2}\end{matrix}\right]
\left[\begin{matrix}\frac{A}{2} + \frac{B}{2}\\\frac{A}{2} + \frac{B}{2}\end{matrix}\right]


Verify
\begin{align}
    P_L^2 & = P_L,\\
    P_R^2 & = P_R,\\
    P_L P_R & = 0.
\end{align}

In [10]:
pL*pL, pL, pR*pR, pR, pL*pR

⎛⎡1/2   -1/2⎤  ⎡1/2   -1/2⎤  ⎡1/2  1/2⎤  ⎡1/2  1/2⎤  ⎡0  0⎤⎞
⎜⎢          ⎥, ⎢          ⎥, ⎢        ⎥, ⎢        ⎥, ⎢    ⎥⎟
⎝⎣-1/2  1/2 ⎦  ⎣-1/2  1/2 ⎦  ⎣1/2  1/2⎦  ⎣1/2  1/2⎦  ⎣0  0⎦⎠

## Decomposition of Dirac spinors

### Decomposition into left and right chiral components
Consider the spinors
\begin{align}
    u_r(p) & = \sqrt{E + m} \begin{pmatrix} \phi_r \\ \frac{\mathbf{\sigma} \cdot \mathbf{p}}{E + m} \phi_r \end{pmatrix},\\
  v_r(p) & = \sqrt{E + m} \begin{pmatrix} \frac{\mathbf{\sigma} \cdot \mathbf{p}}{E + m} \chi_r \\ \chi_r \end{pmatrix},
\end{align}
for particles and antiparticles, respectively. Given a spinor

\begin{align}
\left[\begin{matrix}A\\B\end{matrix}\right],
\end{align}
its left and right-handed components are given by (see above)

\begin{align}
\frac{1}{2}\left[\begin{matrix} A - B \\- A + B\end{matrix}\right] &\text{ and }
\frac{1}{2}\left[\begin{matrix} A + B \\  A + B\end{matrix}\right],
\end{align}

respectively.

Applying these results to $u$ yields the quantities, 

\begin{align}
u_L & = \frac{\sqrt{E+m}}{2}\begin{pmatrix} +(1 - \frac{\sigma \cdot \mathbf{p}}{E+m} )\phi_r \\ -(1 - \frac{\sigma \cdot \mathbf{p}}{E+m}) \phi_r \end{pmatrix}, \quad
u_R  = \frac{\sqrt{E+m}}{2}\begin{pmatrix} +(1 + \frac{\sigma \cdot \mathbf{p}}{E+m} )\phi_r \\ +(1 + \frac{\sigma \cdot \mathbf{p}}{E+m}) \phi_r \end{pmatrix} ,
\end{align}

that is,  left and right chiral components of the Dirac spinor $u$. 

### Decomposition into left and right helicity components

Each chiral component can be decomposed into helicity states. Define 

\begin{align}
P_\pm & = \frac{1}{2} \left(1 \pm \frac{\sigma\cdot\mathbf{p}}{p}\right),\\
      & = \frac{1}{2} \left(1 \pm \sigma\cdot\mathbf{\hat{p}}\right),
\end{align}
which, noting that $(\sigma\cdot\mathbf{\hat{p}})^2=1$, satisfy the relations

\begin{align}
    P_+ + P_- & = 1, \quad P_+ P_- = 0, \quad P_\pm^2 = P_\pm, \\
    P_+ - P_- & = \sigma\cdot\mathbf{\hat{p}}, \\
    \sigma\cdot\mathbf{\hat{p}} P_\pm & = \pm P_\pm.
\end{align}

Using the identity
\begin{align}
1 \pm a A & = \frac{(1-a)}{2} (1 \mp A) + \frac{(1+a)}{2}(1 \pm A),
\end{align}
we can decompose the left and right spinors as follows,

\begin{align}
 u_L & = (1 - a) \frac{\sqrt{E+m}}{2}\begin{pmatrix} +P_+ \phi \\ -P_+ \phi \end{pmatrix} + (1 + a) \frac{\sqrt{E+m}}{2} \begin{pmatrix} +P_- \phi \\ -P_- \phi \end{pmatrix},\\
u_R & = (1 - a) \frac{\sqrt{E+m}}{2} \begin{pmatrix} +P_- \phi \\ + P_- \phi \end{pmatrix} + (1 + a) \frac{\sqrt{E+m}}{2} \begin{pmatrix} +P_+ \phi \\ +P_+ \phi \end{pmatrix},
\end{align}
where $$a = \frac{p}{E + m} = \frac{\beta}{1 + \gamma^{-1}}, \quad\gamma=1 / \sqrt{1 - \beta^2}.$$
In the ultra-relativistic limit $a \rightarrow 1$. Therefore, for both $u_L$ and $u_R$ the first terms vanish. We see that the left chirality state $u_L$ becomes identical to a left helicity state, while $u_R$ becomes indistinguishable from a right helicity state.


We can also decompose the spinor $u$ into positive and negative helicity spinors:

\begin{align}
u & = \sqrt{E+m}\begin{pmatrix} P_+ \phi \\ a P_+ \phi \end{pmatrix} + 
\sqrt{E+m}\begin{pmatrix} P_- \phi \\ -a P_- \phi \end{pmatrix}.
\end{align}


### $v_L$ and $v_R$ spinor helicity decomposition

The helicity operator for an antiparticle is taken to be the negative of that of a particle. Consequently, when the operator is applied to a right helicity state, one obtains a negative eigenvalue. Likewise, when the operator is applied to a left helicity state one obtains a positive eigenvalue. 

Following a similar procedure for $v$ as followed for the spinor $u$ leads to the helicity decomposition

\begin{align}
v & = \sqrt{E+m}\begin{pmatrix} a P_+ \chi \\ P_+ \chi \end{pmatrix} + 
\sqrt{E+m}\begin{pmatrix} -a P_- \chi \\ P_- \chi \end{pmatrix},
\end{align}
where the first term is proportional to $v_\downarrow$ and the second term proportional to $v_\uparrow$.

### 2-component helicity spinors in Cartesian coordinates
For $P_\pm \phi$, we are free to use either $\phi_1 = \begin{pmatrix} 1 \\ 0 \end{pmatrix}$ or $\phi_2 = \begin{pmatrix} 0 \\ 1 \end{pmatrix}$. 

### Define Dirac helicity spinors
We'll follow the usual convention and choose 
\begin{align}
    u_{\uparrow 1}   & = P_+ \phi_1,\\
    u_{\downarrow 1} & = P_- \phi_1.
\end{align}
Then we can write
\begin{align}
& \textbf{particles} \nonumber\\
u_{\uparrow} & = \sqrt{E+m} \begin{pmatrix} u_{\uparrow 1} \\ a u_{\uparrow 1} \end{pmatrix},\quad
u_{\downarrow} = \sqrt{E+m} \begin{pmatrix} u_{\downarrow 1} \\ -a u_{\downarrow 1} \end{pmatrix},
\end{align}

\begin{align}
& \textbf{antiparticles} \nonumber\\
v_{\downarrow} & = \sqrt{E+m} \begin{pmatrix} a u_{\uparrow 1} \\ u_{\uparrow 1} \end{pmatrix},\quad
v_{\uparrow} = \sqrt{E+m} \begin{pmatrix} -a u_{\downarrow 1} \\ u_{\downarrow 1} \end{pmatrix},
\end{align}

where the spinors have been normalized to $2E$.


Check $(\sigma\cdot\mathbf{\hat{p}})^2=1$

In [11]:
sigmaphat2 = sigmaphat**2
sigmaphat2.simplify()
sigmaphat2 = sigmaphat2.subs({px**2+py**2+pz**2: p**2})
sigmaphat2

⎡1  0⎤
⎢    ⎥
⎣0  1⎦

Define,
\begin{align}
    P_+ & = \frac{1}{2}(1 + \sigma\cdot\mathbf{\hat{p}}),\\
    P_- & = \frac{1}{2}(1 - \sigma\cdot\mathbf{\hat{p}}),\\
    u_{\uparrow 1} & = P_+ \phi_1, \\
    u_{\uparrow 2} & = P_+ \phi_2, \\
    u_{\downarrow 1} & = P_- \phi_1, \\
    u_{\downarrow 2} & = P_- \phi_2 ,
\end{align}
in Cartesian coordinates.

In [12]:
pplus = (one + sigmaphat)/2
pminus= (one - sigmaphat)/2

up1   = pplus  * Phi_1
up2   = pplus  * Phi_2

ud1   = pminus * Phi_1
ud2   = pminus * Phi_2

up1, up2, ud1, ud2

⎛⎡ 1   p_z  ⎤  ⎡pₓ - ⅈ⋅p_y⎤  ⎡   1   p_z    ⎤  ⎡-(pₓ - ⅈ⋅p_y) ⎤⎞
⎜⎢ ─ + ───  ⎥  ⎢──────────⎥  ⎢   ─ - ───    ⎥  ⎢──────────────⎥⎟
⎜⎢ 2   2⋅p  ⎥  ⎢   2⋅p    ⎥  ⎢   2   2⋅p    ⎥  ⎢     2⋅p      ⎥⎟
⎜⎢          ⎥, ⎢          ⎥, ⎢              ⎥, ⎢              ⎥⎟
⎜⎢pₓ + ⅈ⋅p_y⎥  ⎢ 1   p_z  ⎥  ⎢-(pₓ + ⅈ⋅p_y) ⎥  ⎢   1   p_z    ⎥⎟
⎜⎢──────────⎥  ⎢ ─ - ───  ⎥  ⎢──────────────⎥  ⎢   ─ + ───    ⎥⎟
⎝⎣   2⋅p    ⎦  ⎣ 2   2⋅p  ⎦  ⎣     2⋅p      ⎦  ⎣   2   2⋅p    ⎦⎠

### 2-component helicity spinors in spherical polar coordinates

Define some substitution maps.

In [13]:
theta, fi = sm.symbols('\\theta, \phi', real=True)

subp      = {px**2+py**2+pz**2:p**2}

# to polar coordinates
subpolar  = {px:p*sm.sin(theta)*sm.cos(fi), 
             py:p*sm.sin(theta)*sm.sin(fi), 
             pz:p*sm.cos(theta)}

# to half-angles
subhangle = {sm.sin(theta): 2*sm.sin(theta/2)*sm.cos(theta/2),
             sm.cos(theta): 2*sm.cos(theta/2)**2 - 1}

# cos(phi) + i*sin(phi) => exp(i*phi)
subc   = {I*sm.sin(fi) + sm.cos(fi): sm.exp(I*fi)}

# cos(phi) - i*sin(phi) => exp(-i*phi)
subcc  = {-I*sm.sin(fi) + sm.cos(fi): sm.exp(-I*fi)}

# get rid of absolute values
subrmabs = {sm.Abs(sm.sin(theta/2)): sm.sin(theta/2), 
            sm.Abs(sm.cos(theta/2)): sm.cos(theta/2)}

  1. Compute $u^\dagger u$. (conjugate method causes problems)
  1. Convert to spinors to spherical polar coordinates

In [14]:
def norm(u):
    uT    = u.transpose()
    uT[0] = uT[0].subs({I: -I})  # I => -I
    uT[1] = uT[1].subs({I: -I})
    return sm.trigsimp((uT*u)[0])

def to_polar(u):
    
    # convert to polar coordinates
    u_s    = u.subs(subpolar)

    # 1. convert to half-angles 
    # 2. convert cos(phi) + i*sin(phi) to exp(i*phi) form etc.
    # 3. do some trig simplification
    u_s[0] = sm.trigsimp(u_s[0].subs(subhangle)).subs(subc).subs(subcc)
    u_s[1] = sm.trigsimp(u_s[1].subs(subhangle)).subs(subc).subs(subcc)
    
    # normalize spinor
    N      = sm.sqrt(norm(u_s)).factor(sm.sqrt(2))
    u_s    = u_s / N
    
    # back to half-angles and simplify!
    u_s[0] = sm.trigsimp(u_s[0]).subs(subhangle).simplify()
    u_s[1] = sm.trigsimp(u_s[1]).subs(subhangle).simplify()
    
    # get rid of absolute value
    u_s[0] = u_s[0].subs(subrmabs).trigsimp()
    u_s[1] = u_s[1].subs(subrmabs).trigsimp()
    return u_s

def simplify_spinor(u):
    for i in range(2):
        u[i] = u[i].subs(subhangle).expand()
        u[i] = u[i].trigsimp()

# convert spinors to polar coordinates
up1_s = to_polar(up1)
ud1_s = to_polar(ud1)

up2_s = to_polar(up2)
ud2_s = to_polar(ud2)

simplify_spinor(up1_s)
simplify_spinor(ud1_s)
simplify_spinor(up2_s)
simplify_spinor(ud2_s)

# verify that we get the conventional form of the helicity spinors
up1_s, ud1_s, up2_s, ud2_s

⎛⎡       ⎛\theta⎞    ⎤  ⎡       ⎛\theta⎞     ⎤  ⎡ -ⅈ⋅\phi    ⎛\theta⎞⎤  ⎡  -ⅈ⋅
⎜⎢    cos⎜──────⎟    ⎥  ⎢    sin⎜──────⎟     ⎥  ⎢ℯ       ⋅cos⎜──────⎟⎥  ⎢-ℯ   
⎜⎢       ⎝  2   ⎠    ⎥  ⎢       ⎝  2   ⎠     ⎥  ⎢            ⎝  2   ⎠⎥  ⎢     
⎜⎢                   ⎥, ⎢                    ⎥, ⎢                    ⎥, ⎢     
⎜⎢ ⅈ⋅\phi    ⎛\theta⎞⎥  ⎢  ⅈ⋅\phi    ⎛\theta⎞⎥  ⎢       ⎛\theta⎞     ⎥  ⎢     
⎜⎢ℯ      ⋅sin⎜──────⎟⎥  ⎢-ℯ      ⋅cos⎜──────⎟⎥  ⎢    sin⎜──────⎟     ⎥  ⎢     
⎝⎣           ⎝  2   ⎠⎦  ⎣            ⎝  2   ⎠⎦  ⎣       ⎝  2   ⎠     ⎦  ⎣     

\phi    ⎛\theta⎞⎤⎞
    ⋅sin⎜──────⎟⎥⎟
        ⎝  2   ⎠⎥⎟
                ⎥⎟
   ⎛\theta⎞     ⎥⎟
cos⎜──────⎟     ⎥⎟
   ⎝  2   ⎠     ⎦⎠

Convert $\sigma\cdot\mathbf{\hat{p}}$ to polar coordinates.

In [15]:
sigma_p    = sigmaphat.subs(subpolar)
sigma_p[0] = sigma_p[0].simplify()
sigma_p[1] = sigma_p[1].simplify()
sigma_p[2] = sigma_p[2].simplify()
sigmaphat, sigma_p

⎛⎡   p_z      pₓ - ⅈ⋅p_y⎤                                             ⎞
⎜⎢   ───      ──────────⎥  ⎡                      -ⅈ⋅\phi            ⎤⎟
⎜⎢    p           p     ⎥  ⎢    cos(\theta)      ℯ       ⋅sin(\theta)⎥⎟
⎜⎢                      ⎥, ⎢                                         ⎥⎟
⎜⎢pₓ + ⅈ⋅p_y    -p_z    ⎥  ⎢ ⅈ⋅\phi                                  ⎥⎟
⎜⎢──────────    ─────   ⎥  ⎣ℯ      ⋅sin(\theta)      -cos(\theta)    ⎦⎟
⎝⎣    p           p     ⎦                                             ⎠

Verify that $u_{\uparrow 1}$, $u_{\uparrow 2}$ and indeed eigen-spinors of $\sigma\cdot\mathbf{\hat{p}}$ with eigenvalue $+1$, while $u_{\downarrow 1}$, $u_{\downarrow 2}$ have eigenvalue $-1$.

In [16]:
y = sigma_p * up1_s
simplify_spinor(y)
up1_s, y

⎛⎡       ⎛\theta⎞    ⎤  ⎡       ⎛\theta⎞    ⎤⎞
⎜⎢    cos⎜──────⎟    ⎥  ⎢    cos⎜──────⎟    ⎥⎟
⎜⎢       ⎝  2   ⎠    ⎥  ⎢       ⎝  2   ⎠    ⎥⎟
⎜⎢                   ⎥, ⎢                   ⎥⎟
⎜⎢ ⅈ⋅\phi    ⎛\theta⎞⎥  ⎢ ⅈ⋅\phi    ⎛\theta⎞⎥⎟
⎜⎢ℯ      ⋅sin⎜──────⎟⎥  ⎢ℯ      ⋅sin⎜──────⎟⎥⎟
⎝⎣           ⎝  2   ⎠⎦  ⎣           ⎝  2   ⎠⎦⎠

In [17]:
y = sigma_p * ud1_s
simplify_spinor(y)
ud1_s, -y

⎛⎡       ⎛\theta⎞     ⎤  ⎡       ⎛\theta⎞     ⎤⎞
⎜⎢    sin⎜──────⎟     ⎥  ⎢    sin⎜──────⎟     ⎥⎟
⎜⎢       ⎝  2   ⎠     ⎥  ⎢       ⎝  2   ⎠     ⎥⎟
⎜⎢                    ⎥, ⎢                    ⎥⎟
⎜⎢  ⅈ⋅\phi    ⎛\theta⎞⎥  ⎢  ⅈ⋅\phi    ⎛\theta⎞⎥⎟
⎜⎢-ℯ      ⋅cos⎜──────⎟⎥  ⎢-ℯ      ⋅cos⎜──────⎟⎥⎟
⎝⎣            ⎝  2   ⎠⎦  ⎣            ⎝  2   ⎠⎦⎠

In [18]:
y = sigma_p * up2_s
simplify_spinor(y)
up2_s, y

⎛⎡ -ⅈ⋅\phi    ⎛\theta⎞⎤  ⎡ -ⅈ⋅\phi    ⎛\theta⎞⎤⎞
⎜⎢ℯ       ⋅cos⎜──────⎟⎥  ⎢ℯ       ⋅cos⎜──────⎟⎥⎟
⎜⎢            ⎝  2   ⎠⎥  ⎢            ⎝  2   ⎠⎥⎟
⎜⎢                    ⎥, ⎢                    ⎥⎟
⎜⎢       ⎛\theta⎞     ⎥  ⎢       ⎛\theta⎞     ⎥⎟
⎜⎢    sin⎜──────⎟     ⎥  ⎢    sin⎜──────⎟     ⎥⎟
⎝⎣       ⎝  2   ⎠     ⎦  ⎣       ⎝  2   ⎠     ⎦⎠

In [19]:
y    = sigma_p * ud2_s
simplify_spinor(y)
ud2_s, -y

⎛⎡  -ⅈ⋅\phi    ⎛\theta⎞⎤  ⎡  -ⅈ⋅\phi    ⎛\theta⎞⎤⎞
⎜⎢-ℯ       ⋅sin⎜──────⎟⎥  ⎢-ℯ       ⋅sin⎜──────⎟⎥⎟
⎜⎢             ⎝  2   ⎠⎥  ⎢             ⎝  2   ⎠⎥⎟
⎜⎢                     ⎥, ⎢                     ⎥⎟
⎜⎢        ⎛\theta⎞     ⎥  ⎢        ⎛\theta⎞     ⎥⎟
⎜⎢     cos⎜──────⎟     ⎥  ⎢     cos⎜──────⎟     ⎥⎟
⎝⎣        ⎝  2   ⎠     ⎦  ⎣        ⎝  2   ⎠     ⎦⎠

Choose which 2-component spinors to use

In [20]:
phi_up = up1_s
phi_dn = ud1_s
phi_up, phi_dn

⎛⎡       ⎛\theta⎞    ⎤  ⎡       ⎛\theta⎞     ⎤⎞
⎜⎢    cos⎜──────⎟    ⎥  ⎢    sin⎜──────⎟     ⎥⎟
⎜⎢       ⎝  2   ⎠    ⎥  ⎢       ⎝  2   ⎠     ⎥⎟
⎜⎢                   ⎥, ⎢                    ⎥⎟
⎜⎢ ⅈ⋅\phi    ⎛\theta⎞⎥  ⎢  ⅈ⋅\phi    ⎛\theta⎞⎥⎟
⎜⎢ℯ      ⋅sin⎜──────⎟⎥  ⎢-ℯ      ⋅cos⎜──────⎟⎥⎟
⎝⎣           ⎝  2   ⎠⎦  ⎣            ⎝  2   ⎠⎦⎠

Define 

$
\begin{align}
s & = (p_1 + p_2)^2,\\
N & = \sqrt{E + m}, \\ 
  & = \sqrt{E} \sqrt{1 + \sqrt{1 - \beta^2}}, \\
a & = \frac{p}{E + m},\\ 
  & = \frac{\beta} {1 + \sqrt{1 - \beta^2}}.
\end{align}
$

In [21]:
s, beta  = sm.symbols('s, \\beta')
a = beta/(1+sm.sqrt(1-beta**2))
N = sm.sqrt(E)*sm.sqrt(1 + sm.sqrt(1-beta**2))
N, a

⎛       _____________________                     ⎞
⎜      ╱    ____________                          ⎟
⎜     ╱    ╱          2               \beta       ⎟
⎜√E⋅╲╱   ╲╱  1 - \beta   + 1 , ───────────────────⎟
⎜                                 ____________    ⎟
⎜                                ╱          2     ⎟
⎝                              ╲╱  1 - \beta   + 1⎠

Construct Dirac helicity spinors for particles

In [22]:
u_up = N*Matrix([[phi_up], [ a*phi_up]])
u_dn = N*Matrix([[phi_dn], [-a*phi_dn]])
u_up, u_dn

⎛⎡           _____________________                ⎤  ⎡           _____________
⎜⎢          ╱    ____________                     ⎥  ⎢          ╱    _________
⎜⎢         ╱    ╱          2          ⎛\theta⎞    ⎥  ⎢         ╱    ╱         
⎜⎢    √E⋅╲╱   ╲╱  1 - \beta   + 1 ⋅cos⎜──────⎟    ⎥  ⎢    √E⋅╲╱   ╲╱  1 - \bet
⎜⎢                                    ⎝  2   ⎠    ⎥  ⎢                        
⎜⎢                                                ⎥  ⎢                        
⎜⎢       _____________________                    ⎥  ⎢        ________________
⎜⎢      ╱    ____________                         ⎥  ⎢       ╱    ____________
⎜⎢     ╱    ╱          2        ⅈ⋅\phi    ⎛\theta⎞⎥  ⎢      ╱    ╱          2 
⎜⎢√E⋅╲╱   ╲╱  1 - \beta   + 1 ⋅ℯ      ⋅sin⎜──────⎟⎥  ⎢-√E⋅╲╱   ╲╱  1 - \beta  
⎜⎢                                        ⎝  2   ⎠⎥  ⎢                        
⎜⎢                                                ⎥  ⎢                        
⎜⎢                          ⎛\theta⎞              ⎥ 

Construct Dirac helicity spinors for antiparticles

In [23]:
v_dn = N*Matrix([[ a*phi_up], [phi_up]])
v_up = N*Matrix([[-a*phi_dn], [phi_dn]])
v_up, v_dn

⎛⎡                           ⎛\theta⎞              ⎤  ⎡                       
⎜⎢              -√E⋅\beta⋅sin⎜──────⎟              ⎥  ⎢              √E⋅\beta⋅
⎜⎢                           ⎝  2   ⎠              ⎥  ⎢                       
⎜⎢            ─────────────────────────            ⎥  ⎢           ────────────
⎜⎢                _____________________            ⎥  ⎢               ________
⎜⎢               ╱    ____________                 ⎥  ⎢              ╱    ____
⎜⎢              ╱    ╱          2                  ⎥  ⎢             ╱    ╱    
⎜⎢            ╲╱   ╲╱  1 - \beta   + 1             ⎥  ⎢           ╲╱   ╲╱  1 -
⎜⎢                                                 ⎥  ⎢                       
⎜⎢                    ⅈ⋅\phi    ⎛\theta⎞           ⎥  ⎢                    ⅈ⋅\
⎜⎢          √E⋅\beta⋅ℯ      ⋅cos⎜──────⎟           ⎥  ⎢          √E⋅\beta⋅ℯ   
⎜⎢                              ⎝  2   ⎠           ⎥  ⎢                       
⎜⎢          ────────────────────────────           ⎥

###  $e^+ e^- \rightarrow \mu^+ \mu^-$ cross section

The differential cross section (in the center of mass) for the process $1, 2 \rightarrow 3, 4$ is given by

$
\begin{align}
    \frac{d\sigma}{d\Omega} & = \frac{1}{64\pi^2 s}\frac{|\mathbf{p}_3|}{|\mathbf{p}_1|} \langle |M|^2 \rangle, \nonumber\\
    & = \frac{1}{64\pi^2 s} \langle |M|^2 \rangle .
\end{align}
$

The 4-momenta are
\begin{align}
    p_1 & = E(1, 0, 0, \beta_1), \tag{$e^-$}\\
    p_2 & = E(1, 0, 0,-\beta_1), \tag{$e^+$}\\
    p_3 & = E(1, \beta_3 \sin\theta, 0, \beta_3 \cos\theta), \tag{$\mu^-$}\\
    p_4 & = E(1,-\beta_3 \sin\theta, 0,-\beta_3 \cos\theta), \tag{$\mu^+$}
\end{align}

Without loss of generality, we can take $\phi = 0$.

### Matrix elements
Note: $e^2 = 4\pi \alpha$. We need to compute matrix elements of the form 

\begin{align}
    M & = \frac{4\pi\alpha}{s} [\overline{v}(p_2) \gamma^\mu u(p_1)]
    [\overline{u}(p_3) \gamma_\mu v(p_4)].
\end{align}

There are 4 helicity state combinations in the initial state and 4 in the final state. Therefore, in principle, there are 16 non-interfering processes and we need a matrix element for each. The processes do not interfere because the helicity states are orthogonal.

We now insert the specific 4-momenta into the helicity spinors. 

In [24]:
beta1, beta3 = sm.symbols('\\beta_1, \\beta_3')
def subs_spinor(u, Beta, Fi, Theta=None):
    u_s = u.subs({beta: Beta, fi: Fi})
    if Theta == None: return u_s
    if Theta in [0, sm.pi, -sm.pi]:
        for i in range(4):
            u_s[i] = sm.limit(u_s[i], theta, Theta)
    else:
        u_s = u_s.subs({theta: Theta})
    return u_s

$e^-$

$
\begin{align}
    p_1 & = E(1, 0, 0, \beta_1)
\end{align}
$

In [25]:
u_up_1 = subs_spinor(u_up, Beta=beta1, Fi=0, Theta=0)
u_dn_1 = subs_spinor(u_dn, Beta=beta1, Fi=0, Theta=0)
u_up_1, u_dn_1

⎛⎡       ______________________⎤  ⎡              0               ⎤⎞
⎜⎢      ╱    _____________     ⎥  ⎢                              ⎥⎟
⎜⎢     ╱    ╱           2      ⎥  ⎢        ______________________⎥⎟
⎜⎢√E⋅╲╱   ╲╱  1 - \beta₁   + 1 ⎥  ⎢       ╱    _____________     ⎥⎟
⎜⎢                             ⎥  ⎢      ╱    ╱           2      ⎥⎟
⎜⎢              0              ⎥  ⎢-√E⋅╲╱   ╲╱  1 - \beta₁   + 1 ⎥⎟
⎜⎢                             ⎥  ⎢                              ⎥⎟
⎜⎢         √E⋅\beta₁           ⎥, ⎢              0               ⎥⎟
⎜⎢ ──────────────────────────  ⎥  ⎢                              ⎥⎟
⎜⎢     ______________________  ⎥  ⎢          √E⋅\beta₁           ⎥⎟
⎜⎢    ╱    _____________       ⎥  ⎢  ──────────────────────────  ⎥⎟
⎜⎢   ╱    ╱           2        ⎥  ⎢      ______________________  ⎥⎟
⎜⎢ ╲╱   ╲╱  1 - \beta₁   + 1   ⎥  ⎢     ╱    _____________       ⎥⎟
⎜⎢                             ⎥  ⎢    ╱    ╱           2        ⎥⎟
⎝⎣              0              ⎦  ⎣  ╲╱   ╲╱  1 

$e^+$

$
\begin{align}
    p_2 & = E(1, 0, 0,-\beta_1)
\end{align}
$

In [26]:
v_up_2 = subs_spinor(v_up, Beta=beta1, Fi=0, Theta=-sm.pi)
v_dn_2 = subs_spinor(v_dn, Beta=beta1, Fi=0, Theta=-sm.pi)
v_up_2, v_dn_2

⎛⎡          √E⋅\beta₁           ⎤  ⎡              0               ⎤⎞
⎜⎢  ──────────────────────────  ⎥  ⎢                              ⎥⎟
⎜⎢      ______________________  ⎥  ⎢         -√E⋅\beta₁           ⎥⎟
⎜⎢     ╱    _____________       ⎥  ⎢  ──────────────────────────  ⎥⎟
⎜⎢    ╱    ╱           2        ⎥  ⎢      ______________________  ⎥⎟
⎜⎢  ╲╱   ╲╱  1 - \beta₁   + 1   ⎥  ⎢     ╱    _____________       ⎥⎟
⎜⎢                              ⎥  ⎢    ╱    ╱           2        ⎥⎟
⎜⎢              0               ⎥, ⎢  ╲╱   ╲╱  1 - \beta₁   + 1   ⎥⎟
⎜⎢                              ⎥  ⎢                              ⎥⎟
⎜⎢        ______________________⎥  ⎢              0               ⎥⎟
⎜⎢       ╱    _____________     ⎥  ⎢                              ⎥⎟
⎜⎢      ╱    ╱           2      ⎥  ⎢        ______________________⎥⎟
⎜⎢-√E⋅╲╱   ╲╱  1 - \beta₁   + 1 ⎥  ⎢       ╱    _____________     ⎥⎟
⎜⎢                              ⎥  ⎢      ╱    ╱           2      ⎥⎟
⎝⎣              0               ⎦ 

$\mu^-$

$
\begin{align}
    p_3 & = E(1, \beta_3 \sin\theta, 0, \beta_3 \cos\theta)
\end{align}
$

In [27]:
u_up_3 = subs_spinor(u_up, Beta=beta3, Fi=0)
u_dn_3 = subs_spinor(u_dn, Beta=beta3, Fi=0)
u_up_3, u_dn_3

⎛⎡       ______________________            ⎤  ⎡       ______________________  
⎜⎢      ╱    _____________                 ⎥  ⎢      ╱    _____________       
⎜⎢     ╱    ╱           2          ⎛\theta⎞⎥  ⎢     ╱    ╱           2        
⎜⎢√E⋅╲╱   ╲╱  1 - \beta₃   + 1 ⋅cos⎜──────⎟⎥  ⎢√E⋅╲╱   ╲╱  1 - \beta₃   + 1 ⋅s
⎜⎢                                 ⎝  2   ⎠⎥  ⎢                               
⎜⎢                                         ⎥  ⎢                               
⎜⎢       ______________________            ⎥  ⎢        ______________________ 
⎜⎢      ╱    _____________                 ⎥  ⎢       ╱    _____________      
⎜⎢     ╱    ╱           2          ⎛\theta⎞⎥  ⎢      ╱    ╱           2       
⎜⎢√E⋅╲╱   ╲╱  1 - \beta₃   + 1 ⋅sin⎜──────⎟⎥  ⎢-√E⋅╲╱   ╲╱  1 - \beta₃   + 1 ⋅
⎜⎢                                 ⎝  2   ⎠⎥  ⎢                               
⎜⎢                                         ⎥  ⎢                               
⎜⎢                      ⎛\theta⎞           ⎥  ⎢     

$\mu^+$

$
\begin{align}
    p_4 & = E(1,-\beta_3 \sin\theta, 0,-\beta_3 \cos\theta)
\end{align}
$

In [28]:
v_up_4 = subs_spinor(v_up, Beta=beta3, Fi=0, Theta=theta+sm.pi)
v_dn_4 = subs_spinor(v_dn, Beta=beta3, Fi=0, Theta=theta+sm.pi)
v_up_4, v_dn_4

⎛⎡                      ⎛\theta⎞           ⎤  ⎡                       ⎛\theta⎞
⎜⎢        -√E⋅\beta₃⋅cos⎜──────⎟           ⎥  ⎢         -√E⋅\beta₃⋅sin⎜──────⎟
⎜⎢                      ⎝  2   ⎠           ⎥  ⎢                       ⎝  2   ⎠
⎜⎢       ──────────────────────────        ⎥  ⎢        ───────────────────────
⎜⎢           ______________________        ⎥  ⎢            ___________________
⎜⎢          ╱    _____________             ⎥  ⎢           ╱    _____________  
⎜⎢         ╱    ╱           2              ⎥  ⎢          ╱    ╱           2   
⎜⎢       ╲╱   ╲╱  1 - \beta₃   + 1         ⎥  ⎢        ╲╱   ╲╱  1 - \beta₃   +
⎜⎢                                         ⎥  ⎢                               
⎜⎢                      ⎛\theta⎞           ⎥  ⎢                       ⎛\theta⎞
⎜⎢        -√E⋅\beta₃⋅sin⎜──────⎟           ⎥  ⎢          √E⋅\beta₃⋅cos⎜──────⎟
⎜⎢                      ⎝  2   ⎠           ⎥  ⎢                       ⎝  2   ⎠
⎜⎢       ──────────────────────────        ⎥  ⎢     

Compute amplitude $a^\mu_{ij} = \overline{u}_i \gamma^\mu u_j$

In [29]:
def amplitude(u1, u2):
    # compute Dirac conjugate
    u1T= u1.transpose().subs({I: -I})*G0
    
    y0 = sm.trigsimp((u1T*G0*u2)[0].expand()).simplify()
    y1 = sm.trigsimp((u1T*G1*u2)[0].expand()).simplify()
    y2 = sm.trigsimp((u1T*G2*u2)[0].expand()).simplify()
    y3 = sm.trigsimp((u1T*G3*u2)[0].expand()).simplify()
    
    return Matrix([y0,y1,y2,y3])

In [30]:
v2 = [v_up_2, v_dn_2]
u1 = [u_up_1, u_dn_1]

u3 = [u_up_3, u_dn_3]
v4 = [v_up_4, v_dn_4]

In [31]:
print('electron currents')
Je     = []
Jelabel= []
label  = ['up', 'dn']
for i, v in enumerate(v2):
    for j, u in enumerate(u1):
        Jelabel.append('%s(2) * gamma^mu * %s(1)' % (label[i], label[j]))
        Je.append( amplitude(v, u) )
        print('%s' % Jelabel[-1])
        display(Je[-1])
        print('-'*60)
        
print('\nmuon currents')
Jm     = []
Jmlabel= []
for i, u in enumerate(u3):
    for j, v in enumerate(v4):
        Jmlabel.append('%s(3) * gamma_mu * %s(4)' % (label[i], label[j]))
        Jm.append( amplitude(u, v) )
        print('%s' % Jmlabel[-1])
        display(Jm[-1])
        #print('%s %s\n' % (Jmlabel[-1], Jm[-1]))
        print('-'*60)

electron currents
up(2) * gamma^mu * up(1)


⎡                 0                  ⎤
⎢                                    ⎥
⎢                 0                  ⎥
⎢                                    ⎥
⎢                 0                  ⎥
⎢                                    ⎥
⎢    ⎛             _____________    ⎞⎥
⎢    ⎜      2     ╱           2     ⎟⎥
⎢2⋅E⋅⎝\beta₁  - ╲╱  1 - \beta₁   - 1⎠⎥
⎢────────────────────────────────────⎥
⎢           _____________            ⎥
⎢          ╱           2             ⎥
⎣        ╲╱  1 - \beta₁   + 1        ⎦

------------------------------------------------------------
up(2) * gamma^mu * dn(1)


⎡  0   ⎤
⎢      ⎥
⎢ 2⋅E  ⎥
⎢      ⎥
⎢-2⋅ⅈ⋅E⎥
⎢      ⎥
⎣  0   ⎦

------------------------------------------------------------
dn(2) * gamma^mu * up(1)


...

------------------------------------------------------------
dn(3) * gamma_mu * up(4)


⎡       0        ⎤
⎢                ⎥
⎢-2⋅E⋅cos(\theta)⎥
⎢                ⎥
⎢     -2⋅ⅈ⋅E     ⎥
⎢                ⎥
⎣2⋅E⋅sin(\theta) ⎦

------------------------------------------------------------
dn(3) * gamma_mu * dn(4)


⎡                        0                         ⎤
⎢                                                  ⎥
⎢    ⎛               _____________    ⎞            ⎥
⎢    ⎜        2     ╱           2     ⎟            ⎥
⎢2⋅E⋅⎝- \beta₃  + ╲╱  1 - \beta₃   + 1⎠⋅sin(\theta)⎥
⎢──────────────────────────────────────────────────⎥
⎢                  _____________                   ⎥
⎢                 ╱           2                    ⎥
⎢               ╲╱  1 - \beta₃   + 1               ⎥
⎢                                                  ⎥
⎢                        0                         ⎥
⎢                                                  ⎥
⎢    ⎛               _____________    ⎞            ⎥
⎢    ⎜        2     ╱           2     ⎟            ⎥
⎢2⋅E⋅⎝- \beta₃  + ╲╱  1 - \beta₃   + 1⎠⋅cos(\theta)⎥
⎢──────────────────────────────────────────────────⎥
⎢                  _____________                   ⎥
⎢                 ╱           2                    ⎥
⎣               ╲╱  1 - \beta₃   + 1          

------------------------------------------------------------


Compute the 16 matrix elements of the form 
\begin{align}
    M & = \frac{4\pi\alpha}{s} [\overline{v}(p_2) \gamma^\mu u(p_1)]
    [\overline{u}(p_3) \gamma_\mu v(p_4)],
\end{align}
from the above currents.

In [32]:
alpha = sm.symbols('\\alpha')

print('Matrix Elements')
M = []
L = []
n = 0
for i, je in enumerate(Je):
    jeT = je.transpose()
    for j, jm in enumerate(Jm):
        n += 1
        y = (jeT * jm)[0]
        y = 4*sm.pi*alpha*y/s
        
        # do some symbolic simplification
        y = y.expand().simplify().trigsimp()
        
        M.append(y)
        label = Jelabel[i]+' * '+Jmlabel[j]
        L.append(label)
        print('%5d %s' % (n, label))
        display(y)
        print('-'*60)

Matrix Elements
    1 up(2) * gamma^mu * up(1) * up(3) * gamma_mu * up(4)


               ⎛                               _____________                  
      2        ⎜        2       2         2   ╱           2          2        
16⋅π⋅E ⋅\alpha⋅⎝- \beta₁ ⋅\beta₃  + \beta₁ ⋅╲╱  1 - \beta₃   + \beta₁  + \beta
──────────────────────────────────────────────────────────────────────────────
                                                             ⎛   _____________
                                                             ⎜  ╱           2 
                                                           s⋅⎝╲╱  1 - \beta₁  

      _____________                _____________    _____________      _______
 2   ╱           2          2     ╱           2    ╱           2      ╱       
₃ ⋅╲╱  1 - \beta₁   + \beta₃  - ╲╱  1 - \beta₁  ⋅╲╱  1 - \beta₃   - ╲╱  1 - \b
──────────────────────────────────────────────────────────────────────────────
    _____________      _____________      _____________    ⎞                  
   ╱           2      ╱           2      ╱         

------------------------------------------------------------
    2 up(2) * gamma^mu * up(1) * up(3) * gamma_mu * dn(4)


               ⎛               _____________    ⎞            
      2        ⎜        2     ╱           2     ⎟            
16⋅π⋅E ⋅\alpha⋅⎝- \beta₁  + ╲╱  1 - \beta₁   + 1⎠⋅sin(\theta)
─────────────────────────────────────────────────────────────
                     ⎛   _____________    ⎞                  
                     ⎜  ╱           2     ⎟                  
                   s⋅⎝╲╱  1 - \beta₁   + 1⎠                  

------------------------------------------------------------
    3 up(2) * gamma^mu * up(1) * dn(3) * gamma_mu * up(4)


...

------------------------------------------------------------
   15 dn(2) * gamma^mu * dn(1) * dn(3) * gamma_mu * up(4)


               ⎛             _____________    ⎞            
      2        ⎜      2     ╱           2     ⎟            
16⋅π⋅E ⋅\alpha⋅⎝\beta₁  - ╲╱  1 - \beta₁   - 1⎠⋅sin(\theta)
───────────────────────────────────────────────────────────
                    ⎛   _____________    ⎞                 
                    ⎜  ╱           2     ⎟                 
                  s⋅⎝╲╱  1 - \beta₁   + 1⎠                 

------------------------------------------------------------
   16 dn(2) * gamma^mu * dn(1) * dn(3) * gamma_mu * dn(4)


               ⎛                               _____________                  
      2        ⎜        2       2         2   ╱           2          2        
16⋅π⋅E ⋅\alpha⋅⎝- \beta₁ ⋅\beta₃  + \beta₁ ⋅╲╱  1 - \beta₃   + \beta₁  + \beta
──────────────────────────────────────────────────────────────────────────────
                                                             ⎛   _____________
                                                             ⎜  ╱           2 
                                                           s⋅⎝╲╱  1 - \beta₁  

      _____________                _____________    _____________      _______
 2   ╱           2          2     ╱           2    ╱           2      ╱       
₃ ⋅╲╱  1 - \beta₁   + \beta₃  - ╲╱  1 - \beta₁  ⋅╲╱  1 - \beta₃   - ╲╱  1 - \b
──────────────────────────────────────────────────────────────────────────────
    _____________      _____________      _____________    ⎞                  
   ╱           2      ╱           2      ╱         

------------------------------------------------------------


Sum squared matrix elements 
$$\langle |M|^2 \rangle = \frac{1}{4} \sum_{j=1}^{16} M^*_j M_j$$
and average over the initial four helicity states.

In [33]:
MM = (sum([x.subs({I:-I})*x for x in M])).simplify()/4
MM = MM.subs({E: sm.sqrt(s)/2}).trigsimp()
MM

    2       2 ⎛      2       2    2                 2         2    ⎞
16⋅π ⋅\alpha ⋅⎝\beta₁ ⋅\beta₃ ⋅cos (\theta) - \beta₁  - \beta₃  + 3⎠

Finally, compute differential cross section. 

In [34]:
diff_xsec = MM/(64*sm.pi**2*s)
diff_xsec

      2 ⎛      2       2    2                 2         2    ⎞
\alpha ⋅⎝\beta₁ ⋅\beta₃ ⋅cos (\theta) - \beta₁  - \beta₃  + 3⎠
──────────────────────────────────────────────────────────────
                             4⋅s                              

### Relativistic limit

Cross section in relatvistic limit:
$$\frac{d\sigma}{d\Omega} = \frac{\alpha^2}{4s}[1 + \cos^2(\theta)].$$

In [35]:
d_xsec = diff_xsec.subs({beta1:1, beta3:1})
d_xsec

      2 ⎛   2            ⎞
\alpha ⋅⎝cos (\theta) + 1⎠
──────────────────────────
           4⋅s            

Matrix elements in relativistic limit. In this limit, since helicity is conserved all non-helicity conserving amplitudes will be suppressed. 

In [36]:
print('Matrix Elements in Relativistic Limit')

Ms = []
for i, (label, m) in enumerate(zip(L, M)):
    y = m.subs({beta1:1, beta3:1})
    if y.evalf() == 0: continue
    print('-'*60)
    print('%5d %s' % (i+1, label))
    display(y)
    Ms.append(y)

Matrix Elements in Relativistic Limit
------------------------------------------------------------
    6 up(2) * gamma^mu * dn(1) * up(3) * gamma_mu * dn(4)


      2                         
16⋅π⋅E ⋅\alpha⋅(cos(\theta) - 1)
────────────────────────────────
               s                

------------------------------------------------------------
    7 up(2) * gamma^mu * dn(1) * dn(3) * gamma_mu * up(4)


       2                          
-16⋅π⋅E ⋅\alpha⋅(cos(\theta) + 1) 
──────────────────────────────────
                s                 

------------------------------------------------------------
   10 dn(2) * gamma^mu * up(1) * up(3) * gamma_mu * dn(4)


       2                          
-16⋅π⋅E ⋅\alpha⋅(cos(\theta) + 1) 
──────────────────────────────────
                s                 

------------------------------------------------------------
   11 dn(2) * gamma^mu * up(1) * dn(3) * gamma_mu * up(4)


      2                         
16⋅π⋅E ⋅\alpha⋅(cos(\theta) - 1)
────────────────────────────────
               s                

In [37]:
MM = (sum([x.subs({I:-I})*x for x in Ms])).simplify()/4
MM = MM.subs({E: sm.sqrt(s)/2})
MM

    2       2 ⎛   2            ⎞
16⋅π ⋅\alpha ⋅⎝cos (\theta) + 1⎠

In [38]:
diff_xsec = MM/(64*sm.pi**2*s)
diff_xsec

      2 ⎛   2            ⎞
\alpha ⋅⎝cos (\theta) + 1⎠
──────────────────────────
           4⋅s            

### Some useful simplification functions

In [39]:
TR='''
TR0 - simplify expression
TR1 - sec-csc to cos-sin
TR2 - tan-cot to sin-cos ratio
TR2i - sin-cos ratio to tan
TR3 - angle canonicalization
TR4 - functions at special angles
TR5 - powers of sin to powers of cos
TR6 - powers of cos to powers of sin
TR7 - reduce cos power (increase angle)
TR8 - expand products of sin-cos to sums
TR9 - contract sums of sin-cos to products
TR10 - separate sin-cos arguments
TR10i - collect sin-cos arguments
TR11 - reduce double angles
TR12 - separate tan arguments
TR12i - collect tan arguments
TR13 - expand product of tan-cot
TRmorrie - prod(cos(x*2**i), (i, 0, k - 1)) -> sin(2**k*x)/(2**k*sin(x))
TR14 - factored powers of sin or cos to cos or sin power
TR15 - negative powers of sin to cot power
TR16 - negative powers of cos to tan power
TR22 - tan-cot powers to negative powers of sec-csc functions
TR111 - negative sin-cos-tan powers to csc-sec-cot
'''